In [202]:
def read_input(filename):
    data = []
    with open(filename) as input:
        for line in input.readlines():
            splitline1 = line.rsplit('Step ', 1)
            first = splitline1[1][0]
            splitline2 = line.rsplit('step ', 1)
            second = splitline2[1][0]
            data.append((first, second))
    return data
            


In [203]:
def find_prerequisites(data):
    prerequisites = {}
    prerequisite_steps = list(zip(*data))[0]
    for step_prerequisite in data:
        prerequisite, step = step_prerequisite
        prerequisites.setdefault(step, []).append(prerequisite)
        
    prerequisites_copy = {**prerequisites}
    
    for step in prerequisite_steps:
        if step not in prerequisites_copy.keys():
            prerequisites[step] = []
    
    return prerequisites

In [204]:
def get_task_order(prerequisite_dict):
    task_order = []
    while True:
        no_prereq = sorted([key for key, val in prerequisite_dict.items() if not val])
        if not no_prereq:
            break
        task = no_prereq[0]
        task_order.append(task)
        del prerequisites[task]
        for step, prereq  in prerequisite_dict.items():
            if task in prereq:
                prereq.remove(task)
    return ''.join(task_order)

In [209]:
data = read_input('day7input.txt')

In [210]:
prerequisites = find_prerequisites(data)

In [211]:
tasks = get_task_order(prerequisites)


In [212]:
tasks

'JDEKPFABTUHOQSXVYMLZCNIGRW'